<a href="https://colab.research.google.com/github/Husienvora/Machine-learning-project/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🐶 End-to-end Multi-class Dog breed Classification

This notebook builds an end-to-end multi-class image classifier using tensorFlow 2.0 and tensorflow Hub.

##1.Problem

Identifying the breed of a dog given an image of the dog

### A Scenario
Suppose i am sitting in a park and take a photo of a dog of which i want to know the breed of, I will be in need of this model then.

##2. Data
The data we're using is from Kaggle's dog breed identification competition.

https://www.kaggle.com/c/dog-breed-identification/overview

## 3 .Evaluation 
The evaluation is a file for each dog breed of each test image.

https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

##4.Features

Some infromation about the data 
* We're dealing with images(unstructured data) so it's probably best we use deep learning/transfer learning.

* There are 120 breeds of dogs (this means there are 120 different classes).

* There are around 10000+ images in the trainning set(these images have labels)

* There are around 10,000+ images in the test set (these images have no labels ,because we'll want to predict them).

In [ ]:
## Unzip the data.
##!unzip "drive/MyDrive/Dogvision/dog-breed-identification.zip" -d "drive/MyDrive/Dogvision"

## Get our workspace ready

* Import TensorFlow 2.x --Done
* Import TensorFLow Hub --Done
* Make sure we're using a GPU --Done


In [ ]:
# Import necessary tools
import tensorflow as tf
import tensorflow_hub as hub
print("TF version",tf.__version__)
print("TF version",hub.__version__)

#Check for gpu availability 
print("GPU","available(yesss!)" if tf.config.list_physical_devices("GPU") else "not available")



## Getting our data ready (Turnin it into tensors)

## With all machine learning models our data and to be in numerical format so that's what we wiil be doing first turning our iages into tensors(Numerical representation).


In [ ]:
#checkout the labels of our data
import pandas as pd

labels_csv=pd.read_csv("drive/MyDrive/Dogvision/labels.csv")
print(labels_csv.describe())
print(labels_csv.head())

In [ ]:
dataframe_labels_csv=pd.DataFrame({"id":labels_csv["id"],"Breeds":labels_csv["breed"]})
dataframe_labels_csv

In [ ]:
#How many images are there of each breed 
labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

In [ ]:
labels_csv["breed"].value_counts().median()

In [ ]:
#Let's view an image
from IPython.display import Image
Image("drive/MyDrive/Dogvision/train/fff43b07992508bc822f33d8ffd902ae.jpg")

### Getting images and their labels
Let's get a list of all our image file pathnames



In [ ]:
dataframe_labels_csv

In [ ]:
filenames=["drive/MyDrive/Dogvision/train/"+fname +".jpg" for fname in dataframe_labels_csv["id"]]

#Check the first 10
filenames

In [ ]:
# Check wether number of file names matches number of actual image files
import os
if (len(os.listdir("drive/MyDrive/Dogvision/train/"))==len(filenames)):
  print("All the files matched ..proceed")
else:

  print("Check the directory for issues")  

In [ ]:
#One more check
Image(filenames[9000])

In [ ]:
labels_csv["breed"][9000]

Since we've image filepaths in a list lets prepare our labels

In [ ]:
import numpy as np
labels=labels_csv["breed"].to_numpy()
labels

In [ ]:
len(labels)

In [ ]:
#See if number of labels matches number of filenames

if (len(labels)==len(filenames)):
  print("Number of labels matches number of filenames")
else:
  print("Number of labels does not match number of filenames .check the data directories")


In [ ]:
#Find the unique label values
unique_breeds=np.unique(labels)
unique_breeds

In [ ]:
len(unique_breeds)

In [ ]:
# Turn a single label into an array of booleans
#FOR EG-
print(labels[3])
labels[3]==unique_breeds


In [ ]:
#Turn every label int a boolean array
boolean_labels=[label==unique_breeds for label in labels]
boolean_labels[:3]

In [ ]:
len(boolean_labels)

In [ ]:
# Example turning boolean array into integers

print(labels[0])#Original label
print(np.where(unique_breeds==labels[0])) # index where label occurs
print(boolean_labels[0].argmax()) # index where label occurs in boolean array
print(boolean_labels[0].astype(int)) # there will be a 1 where the sample label occurs

### Creatinng our own validation set
Since the dataset from kaggle doesn't come with a validation set ,we're going to create our own


In [ ]:
# Setup X & y variabels
x=filenames
y=boolean_labels


In [ ]:
len(filenames)

We're goin to start off experimenting with ~1000 image and increase as needed


In [ ]:
#Set number of images to use for experimenting 
NUM_IMAGES=1000 #@param {type:"slider",min:1000, max:10000}

In [ ]:
#Let's split our data into train and validation set
from sklearn.model_selection import train_test_split
#split them into trainning and validation of total size NUM_IMAGES
X_train,X_val,y_train,y_val=train_test_split(x[:NUM_IMAGES],
                                             y[:NUM_IMAGES],
                                             test_size=0.2,
                                             random_state=42)
len(X_train),len(y_train),len(X_val),len(y_val)


Let's have a peek at the training data

In [ ]:
Image('drive/MyDrive/Dogvision/train/00bee065dcec471f26394855c5c2f3de.jpg')

In [ ]:

unique_breeds[y_train[0].argmax()]


In [ ]:
X_train[0],y_train[0]

## Preprocessing Images (Turning images into tensors)

To preprocesss out images into tensors we're going to write a function whic does a this things:

   1. Take an image filepath as input
   2. Use TensorFlow to read the file and save it to a variable,image
   3. Turn our image (a jpg)into tensors
   4. Normalize our image (convert color channel value from 0-255 to 0-1
   5. Resize the image to be a shape of (224,224)
   6. Return the modified image

Before we do,let's see what importing an image looks like

In [ ]:
# Convert image to numpy array
from matplotlib.pyplot import imread
image=imread(filenames[42])
image.shape



In [ ]:
image.max(),image.min()

In [ ]:
image[:2]

In [ ]:
tf.constant(image)[:2]

In [ ]:
#Define image size
IMG_SIZE=224

#Create a function for prerocessing images
def process_image(image_path,img_size=IMG_SIZE):
  """
  Takes an image file path and turns the image into tensor
  
  """
  # Read in an image file 
  image=tf.io.read_file(image_path)
  #Turn the jpeg image into numerical tensors with 3 colour channels(Red,green,Blue)
  image=tf.image.decode_jpeg(image,channels=3)
  #Convert te color channels values from 0 to 255 to 0 to 1 values
  image=tf.image.convert_image_dtype(image,tf.float32)
  # Resize the image to our desired value(244,244)
  image=tf.image.resize(image,size=[IMG_SIZE,IMG_SIZE])

  return image




## Turning our data into batches

Why turn our data into batches?

Let's say you're trying to process 10,000+ image in one 
go...they all might not fit into memory.

So that's why we do about 32 (this is the batch size) images at a time (you can manually adjust the batch size if need be).

In order to use Tesnorflow effectively ,we need our data in te form of Tensor tupels which look like this:
`(image,label)`



In [ ]:
#Create a simple function to return a tuple (image,label)

def get_image_label(image_path,label):
  """
  Takes an image path name and the associated label,
  processes the image and return a type of (image,label).
  """
  image=process_image(image_path)
  return image, label
  

In [ ]:
get_image_label(x[42],tf.constant(y[42]))

Now we've got a way to turn our data into tupels of tensors in the form:`(image,label)`.Let's make a function to turn all of our data(x&y) into batches!



In [ ]:
#Define the batch size ,32 is a good start
BATCH_SIZE=32

#Create a function to turn data into batches
def create_data_batches(X,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
  """
  Create batches of data out of image(X) and label (y) pairs.
  Shuffels the data if it's training data bur doesn't shuffle if it's validation data.
  Also accepts test data as input (no labels is set to default value false because test data does not contain labels since we're going to predict those.)
  test data is set to false when batches of training data are being made and set to true when batches of test data are being made.
  """
  # If the data is a test dataset ,we probably don't have labels
  if test_data:
    print("Creating test data batches..")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X))) #Only filepaths (no labels)
    data_batch=data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  # if the data is a valid dataset  ,we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches..")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y))) #(Filepaths,labels)
    data_batch=data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    print("Creating training data batches..")
    #Turn fileapths and labels into Tensors
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))

    #Shuffling pathnames and labels before mapping images processor function is faster than shuffling images
    data =data.shuffle(buffer_size=len(X))

    # Create (image,label) tuples (this also turns the iamge path into a preprocessed image)
    data=data.map(get_image_label)

    #Turn the training data into batches 
    data_batch=data.batch(BATCH_SIZE)

  return data_batch   

  




In [ ]:
#Create training and validation data batches
train_data=create_data_batches(X_train,y_train)
val_data=create_data_batches(X_val,y_val,valid_data=True)

In [ ]:
#Check out the different attributes of our data batches
train_data.element_spec,val_data.element_spec

##Visualizing Data batches

Our data is now in batches ,however these can be a little hard ot understand/comprehend,let's visualize them!

In [ ]:
import matplotlib.pyplot as plt
#Create a function for viewing images in a data batch
def show_25_images (images,labels):
  """
  Displays a plot of 25 images and their labels from a data batch
  """
  #Setup the figure 
  plt.figure(figsize=(10,10))
  #Loop through 25 (for displaying 25 images)
  for i in range(25):
    #Create subplots (5 rows,5columns)
    ax=plt.subplot(5,5,i+1)
    #Display an image
    plt.imshow(images[i])
    #Add the iamge label as the title
    plt.title(unique_breeds[labels[i].argmax()])
    #Turn the grid lines off
    plt.axis("off")
 

In [ ]:

train_images,train_labels=next(train_data.as_numpy_iterator())


plt.imshow(train_images[0]),unique_breeds[train_labels[0].argmax()] 



In [ ]:
#Now let's visualize our data in a training batch
show_25_images(train_images,train_labels)

## Building a model 

Before we build a model ,there are a few things we need to define:
* The input shape (our images shape,in the form of Tensors) to uor model.
* THe output shape (image labelsmin the form of Tensors )of our model
*The URL of the model we want to use from Tesnorflow hub -https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4.

In [ ]:
#Setup input shape to the model
INPUT_SHAPE=[None,IMG_SIZE,IMG_SIZE,3] #batch,height,heightwidthmcoour channels
#Setup output shape of our model
OUTPUT_SHAPE=len(unique_breeds)

#Setup model URL from TensorFlow Hub
MODEL_URL= "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"


Now we have got our inputs ,outputs and model ready to go
Let's put them together into a keras deep learning model

Knowing this,let's create a function which 
* Takes the input shape ,output shape and the model we've chosen as parameters
* Defines the layers in keras model in sequential fashion (do this first,then this,then that).
* Compiles the model (says how it should be evaluated and improved)
* Build the model (tells the model the input shape it'll be getting).
* Returns the model.

All of these steps can be found here:https://www.tensorflow.org/guide/keras/sequential_model

In [ ]:
#Create a function which builds a keras model
def create_model(input_shape=INPUT_SHAPE,output_shape=OUTPUT_SHAPE,model_url=MODEL_URL):
  print("Building model with:",MODEL_URL)

  #Setup the model layers
  model=tf.keras.Sequential([hub.KerasLayer(MODEL_URL),
                             tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                                                   activation="softmax")
  ])
  


  #Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["accuracy"]
  )

  # Build the model
  model.build(INPUT_SHAPE)


  return model


  


In [ ]:
model=create_model()
model.summary()

##Creating callbacks

Callbacks are helper function a model can use during training to do such things as save its progress,check its progress or stop training early if a model stops improving.


we'll createa two callbacks one for tesnorboard which helps track our models progress and another for early stopping
which prevents our model from training too long.

##TensorBoard callback

To setup a Tensorboard callback,we need to do 3 things 
1. load the Tensorboard notebook extension ✔
2. Create a Tesnorboard callback which is able to save logs ✔
3. Visualize our models training logs with `%tensorboard` magic function (we'll do this after model trainning)

In [ ]:
#Load Tensorboard notebook extension
%load_ext tensorboard


In [ ]:
import datetime

#Create a function to build a Tensorboard callback
def create_tensorboard_callback():
  # Create a log directory for storing tensorflow logs
  logdir=os.path.join("drive/MyDrive/Dogvision/logs",
                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)


### Early Stopping callback

Early stopping helps stop our model from overfitting by stopping training if a certain evaluation metric stops imporving

In [ ]:
#Create early stopping callback
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                patience=3)


## Training a model (on subset of data)

Our first model is only going to train on 1000 images,to make sure everything is working

In [ ]:
NUM_EPOCHS =100 #@param {type:"slider",min:10,max:110,step:10}

In [ ]:
#Check to make sure we're still running on a gpu
print('GPU',"available(yes)" if tf.config.list_physical_devices("GPU") else "not available")

Let's create a function which trains a model.

* Create a model using `create_model()`
* Setup a TensorBoard callback using `create_tensorboard_callback()`
* Call the fit() function on our model passing it the training data,number of epochs to train for (`NUM_EPOCHS`) and the callbacks we'd like to use
*Return the model 


In [ ]:
#Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version.
  """
  #Create a model
  model=create_model()

  #Create new Tensorboard session everytime we train our model
  tensorboard = create_tensorboard_callback()
  
  #Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1,
            callbacks=[tensorboard,early_stopping])
  #Return the fitted model
  return model

In [ ]:
#model=train_model()

### Checking the tensorboard logs

The tensorboard magic function (`%tensorboard`) will access the logs directory we created early and visualize it's contents

In [ ]:
%tensorboard --logdir drive/MyDrive/Dogvision/logs

## Making and evaluating prediction using a trained model


In [ ]:
#Make prediction on the validation data (not used to train on)
predictions=model.predict(val_data,verbose=1)
predictions

In [ ]:
predictions.shape

In [ ]:
len(y_val)

In [ ]:
len(unique_breeds)

In [ ]:
np.sum(predictions[0])

In [ ]:
#First prediction
index=81
print(predictions[index])
print(f"Max value (probability of prediction):{np.max(predictions[index])}")
print(f"Sum:{np.sum(predictions[index])}")
print(f"Max index:{np.argmax(predictions[index])}")
print(f"Predicted label:{unique_breeds[np.argmax(predictions[index])]}")

In [ ]:
unique_breeds[37]

Having the above functionality is great but we want to be able to do it at scale.

And it would be even better if we could see the image the predictions is being made on !

**Note!** prediction probabilities as confidence levels



In [ ]:
#Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turn an array of prediction probabilites into a label
  """
  return unique_breeds[np.argmax(prediction_probabilities)]

#Get a predicted label based on array of prediction probabilities
##pred_label=get_pred_label(predictions[81])
#pred_label  


Now since our validation data is still in a batch dataset ,we'll have to unbatchify it to make predictions on the validation images and then compare those predictions to the validation labels(truth labels).

In [ ]:
val_data

In [ ]:
images_=[]
labels_=[]

#Loop through ubatched data
for image,label in val_data.unbatch().as_numpy_iterator():
  images_.append(image)
  labels_.append(label)
  
images_[0],labels_[0]

In [ ]:
get_pred_label(labels_[0])

In [ ]:
get_pred_label(predictions[0])

In [ ]:
#Create a function to unlock a batch dataset
def unbatchify(data):
  """
  Takes a batched dataset if (image,label) Tensors and returns seperate arrays
  of images and labels
  """

  images=[]
  labels=[]

  #Loop through unbatched data
  for image,label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_breeds[np.argmax(label)])
  return images,labels

#Unbatchify the validation data
val_images,val_labels=unbatchify(val_data)
val_images[0],val_labels[0]



In [ ]:
get_pred_label(val_labels[0])

Now we've got ways to get :
* Prediction labels
* Validation labels (truth labels)
* Validation images

Let's make some function to make these all a bit more visual

We'll create a function which:
* Takes an array of prediction probabilities ,an array of truth labels and as array of images and integers ✔
* Convert the prediction probabilites to a predicted label.✔
* Plot the predicted label,its predicted probability ,the truth label and the target image on single plot.✔

In [ ]:
def plot_pred (prediction_probabilities,labels,images,n=77):
  """
  View the prediction ,ground truth and image for sample n
  """
  pred_prob,true_label,image=prediction_probabilities[n],labels[n],images[n]

  #Get the pred label
  pred_label=get_pred_label(pred_prob)

  #Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])
  
  #Change the colour of the title depending on if te predictions is right or wrong
  if pred_label==true_label:
    color="green"
  else:
    color="red"  
  # Change plot title to be predicted ,probability of prediction and truth label
  plt.title("{} {:2.0f}% {}".format(pred_label,
                                    np.max(pred_prob)*100,
                                    true_label),color=color)




In [ ]:
plot_pred(prediction_probabilities=predictions,labels=val_labels,images=val_images)

Now we've got one function to visualize our model top prodection ,let's make another t oview our models top 10 predictions

This function will:
* Take an input of prediction probabilities array and a ground truth array and an integer

* Find the top 10:
 * Prediction probabilites indexes
 * Prediction probabilities values
 * Prediction labels

*  Plot the top 10 prediction probabilities values and labels ,coloring the true label green




In [ ]:
def plot_pred_conf(prediction_probabilities,labels ,n=1):
  """
  plus the top highest predictions confidence along with the truth label for sample n

  """
  pred_prob,true_label=prediction_probabilities[n],labels[n]

  #Get the pred label
  pred_label=get_pred_label(pred_prob)

  #Find the top 10 prediction confidence indexes
  top_10_pred_indexes=pred_prob.argsort()[-10:][::-1]
  
  #Find the top 10 prediction confidence values
  top_10_pred_values=pred_prob[top_10_pred_indexes]

  #Find the top 10 prediction labels
  top_10_pred_labels=unique_breeds[top_10_pred_indexes]

  #Setup plot
  top_plot=plt.bar(np.arange(len(top_10_pred_labels)),
                             top_10_pred_values,
                             color="gray")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")
  #Change color of the true label
  if np.isin(true_label,top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels==true_label)].set_color("green")
  else:
    pass  


In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,n=9)

In [ ]:
predictions[0][predictions[0].argsort()[-10:][::-1]]

In [ ]:
predictions[0].max()


Now we've got some function to help us visualize our predictions and evaluate our model ,let's check out a few.
  

In [ ]:
#Let's check out a few predictions and their different values
i_multiplier=20
num_rows=3
num_cols=2
num_images=num_rows*num_cols
plt.figure(figsize=(10*num_cols,5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows,2*num_cols,2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows,2*num_cols,2*i+2)
  
  plot_pred_conf(prediction_probabilities=predictions,labels=val_labels,n=i+i_multiplier)

plt.tight_layout(h_pad=1.0)  
plt.show()


##Saving and reloading a trained model




In [ ]:
# Create a function to save a model 
def save_model(model,suffix=None) :
  """
  Saves a given model in a models directory and appends a suffix
  """
  #Create a model directory pathname with current time
  modeldir=os.path.join("drive/MyDrive/Dogvision/models",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path=modeldir+"_"+suffix+".h5" #Save format model
  model.save(model_path)
  return model_path
  

In [ ]:
#Create a function to load a trained model
def load_model(model_path):
  """
  Loads a save mdoel from a specific path
  """
  print(f"Loading a saved model from:{model_path}")
  model =tf.keras.models.load_model(model_path,
                                    custom_objects={"KerasLayer":hub.KerasLayer})
  return model
  

Now we've got functions to save and load a trained model ,let's make sure they work

In [ ]:
#Save our model trained on 1000 images
import datetime
save_model(model,suffix="1000-images-mobilenetv2-Adam")

In [ ]:
#Load a trained model
loaded_image_model=load_model("drive/MyDrive/Dogvision/models/20211229-07151640762106_1000-images-mobilenetv2-Adam.h5")

In [ ]:
#Evaluate the pre-save model
model.evaluate(val_data)

In [ ]:
loaded_image_model.evaluate(val_data)

#Training a big dog model 🐶(on the full data)

In [ ]:
len(x),len(y)


In [ ]:
len(X_train)

In [ ]:
#Create a data batch with full data set
full_data = create_data_batches(x,y)

In [ ]:
full_data

In [ ]:
#Create a model for full model
full_model=create_model()

In [ ]:
#Create full model callbacks
full_model_tensorboard=create_tensorboard_callback()

#No validation set when training on all the data ,so we cant monitor validation accuracy
full_model_early_stopping=tf.keras.callbacks.EarlyStopping(monitor="accuracy",patience=3)



**Note:** Running the cell below will take a little while (maybe up to 30 minutes for the first epoch) because the GPU we're using in the runtime has to load all of the images into memory.

In [ ]:
#Fit the model to the full data
##full_model.fit(x=full_data,
              ## callbacks=[full_model_tensorboard,full_model_early_stopping])

In [ ]:
save_model(full_model,suffix="full-image-set-mobilenetv2-Adam")

In [ ]:
#Load in the full model
loaded_full_model=load_model("drive/MyDrive/Dogvision/models/20211229-07181640762301_full-image-set-mobilenetv2-Adam.h5")


In [ ]:
len(x)

#Make predictions on the test data

Since our model has been train on images in the form of Tensor batches ,to make predicitons on the test data,we'll have to get it into the same format.

luckily we created 'create_data_batches()' earlier which can take a list of filenames as input and cover them into Tensor batches

To make predictions on the test data ,we'll ,get the test image filenames.
* Get the test image filenames ✔
* Convert the filenames into test data batches using `create_data_batches` and setting the `test_data` parameter to `True` (since the test data doesn't have labels). ✔
* Make a predictions array by passing the test batches to the `predict()` method called on our model.


In [ ]:
# Load test image filenames
import os
test_path ="/content/drive/MyDrive/Dogvision/test/"
test_filenames=[test_path + fname for fname in os.listdir(test_path)]
test_filenames[0]


In [ ]:
len(test_filenames)

In [ ]:
#Create test data batch
test_data= create_data_batches(test_filenames,test_data=True)

In [ ]:
len(test_data)


**Note** Calling `predict()` on our full data batch using the loaded full model

In [ ]:
# Make predictions on test data batch using the loaded full model 
test_predictions =loaded_full_model.predict(test_data,
                                            verbose=1)

In [ ]:
np.savetxt("drive/MyDrive/Dogvision/preds_array.csv",test_predictions,delimiter=",")

In [ ]:
test_predictions=np.loadtxt("drive/MyDrive/Dogvision/preds_array.csv",delimiter=",")

In [ ]:
len(test_predictions)

In [ ]:
test_predictions.shape


In [ ]:
len(test_predictions)

In [ ]:
images=[]


#Loop through unbatched data
for image in test_data.unbatch().as_numpy_iterator():
  images.append(image)
 

In [ ]:
len(images)

In [ ]:
import PIL
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# Checking if the predictions made on test data set are true you can also cross check them by searching the breed on google

In [ ]:
tensor_to_image(images[10356])

In [ ]:
unique_breeds[test_predictions[10356].argmax()]

## Making predictions on custom images
To make predictions on custom images we'll
* Get the filepath of our own images.
* Turn the filepaths into data batches using `create_data_batches()`.And since our  custom images wont have labels,we set the `test_data` parameter to `True`.
* Pass the custom image data batch to our model's `predict()` method
* Convert the predictions output probabilities to predictions labels
* Compare the predicted labels to the custom images.

In [ ]:
#Get the custom image filepaths
custom_path="drive/MyDrive/Dogvision/custom-dog-photos/"
custom_image_paths=[custom_path+fname for fname in os.listdir(custom_path) ]

In [ ]:
custom_image_paths=custom_image_paths[0:]
custom_image_paths


In [ ]:
#Turn custom images into batch data sets
custom_data=create_data_batches(custom_image_paths,test_data=True)
custom_data


In [ ]:
#Make predictions on the custom data
custom_preds=loaded_full_model.predict(custom_data)

In [ ]:
custom_preds.shape

In [ ]:
custom_pred_labels=[get_pred_label(custom_preds[i]) for i in range(len(custom_preds))]
custom_pred_labels

In [ ]:
images=[]


#Loop through unbatched data
for image in custom_data.unbatch().as_numpy_iterator():
  images.append(image)
 

In [ ]:
custom_pred_labels[2]

In [ ]:
tensor_to_image(images[2])